In [10]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

# Get total list of S&P500 companies
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
with urllib.request.urlopen(url) as response:
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tr_list = table.find_all("tr")

    ticker = []
    company = []
    GICS_sector = []
    GICS_sub_industry = []

    for unit in tr_list[1:]:
        td_list = unit.find_all("td")
        if len(td_list) >= 5:
            company.append(td_list[2].text.strip())            # Company name
            ticker.append(td_list[1].text.strip())             # Ticker symbol
            GICS_sector.append(td_list[3].text.strip())
            GICS_sub_industry.append(td_list[4].text.strip())

    SP500 = {
        'ticker': ticker,
        'company': company,
        'GICS_sector': GICS_sector,
        'GICS_sub_industry': GICS_sub_industry
    }

    SP_df = pd.DataFrame(SP500)
    print(SP_df.head())  # Show only the top rows

    SP_df.to_csv("C:\\Users\\saira\\Desktop\\arima_lstm_model\\SP500_list.csv", index=False)


  ticker                company GICS_sector   GICS_sub_industry
0   COIN               Coinbase         DFS  Discover Financial
1   DASH               DoorDash         BWA          BorgWarner
2    TKO     TKO Group Holdings         TFX            Teleflex
3    WSM  Williams-Sonoma, Inc.          CE            Celanese
4    EXE          Expand Energy         FMC     FMC Corporation


In [12]:
import quandl
import os

API_KEY = ''
start = "2000-01-01"
end = "2017-12-31"
tickers = list(SP_df.ticker)

path = 'C:\\Users\\saira\\Desktop\\arima_lstm_model\\stock_data'
for file in os.listdir(path) :
    os.remove(path+'/'+file)
for item in tickers :
    data = quandl.get("WIKI/{}".format(item.replace(".","_")), start_date=start, end_date=end, api_key=API_KEY)     
    data_dir = "C:\\Users\\saira\\Desktop\\arima_lstm_model\\stock_data"+item+".csv"
    data.to_csv(data_dir)
    
    if os.path.getsize(data_dir) < 250000 :
        print(item+' file size '+str(os.path.getsize(data_dir))+' bytes : reloading data...')
        data = quandl.get("WIKI/{}".format(item.replace(".","_")), start_date=start, end_date=end, api_key=API_KEY)   
        data.to_csv(data_dir)


QuandlError: (Status 403) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@quandl.com.